In [17]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
from datetime import datetime
import glob
from mpl_toolkits.basemap import Basemap, addcyclic
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In this notebook we calculate the ratio between phasespeed errors and zonal wind speed errors to investigate whether these errors are related. 

In [24]:
#set path where annual errors are available
path = '/media/onno/Algemeen/Thesis/GFS_mean/'
#Loop over lead days. Skip day zero as phasespeed errors are undefined on day zero
for i in range(1,11):
    file_error_u_wind = 'GFS_mean_error_u_wind_day_{:02d}_globe_mars.nc'.format(i)
    file_error_phasespeed = 'GFS_mean_error_phasespeed_day_{:02d}_globe_mars.nc'.format(i)
    error_u_wind = xr.open_dataset(path+file_error_u_wind,decode_times=False)
    error_phasespeed = xr.open_dataset(path+file_error_phasespeed,decode_times=False)
    error_u_wind = error_u_wind.where(np.abs(error_u_wind.u)>0.1)
    diff = (error_phasespeed.v /error_u_wind.u).to_dataset(name='ratio')
    diff.to_netcdf(path+'/ratio_cp_u/difference_error_phasespeed_u_wind_day_{:02d}.nc'.format(i))


In [ ]:
#Loop over the two poles
latz_polar=[-90,90]
for lat_polar in latz_polar:
    #create figure
    fig,axz = plt.subplots(2,5,figsize=(20,10))
    #loop over lead days
    for i in range(10):
        ax = axz.flat[i]
        #for each ax set title which day it is
        ax.text(x=0.5,y=1.15,s='Day {}'.format(i),
           transform=ax.transAxes,horizontalalignment='center',fontsize=12)
        ds = xr.open_dataset(path+'/ratio_cp_u/difference_error_phasespeed_u_wind_day_{:02d}.nc'.format(i+1),
                            decode_times=False)
        #add cyclic point for contourf function to work properly
        ds_cyclic, lon_cyclic = addcyclic(ds.ratio.values, ds.lon)
        lon,lat = np.meshgrid(lon_cyclic,ds.lat)

        m = Basemap(projection='ortho',lon_0=0,lat_0=lat_polar,resolution='c',ax=ax)
        im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
                   vmin=-2,vmax=2,levels=np.linspace(-2,2,81))
        #draw coastlines, parellels and meridians
        m.drawcoastlines()
        m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
        m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1])
    #adjust figure slightly to make optimal use of figsize
    fig.subplots_adjust(top=0.85,bottom=0.05,left=0.05,
                        right=0.9,hspace=0.25,wspace=0.25)
    #add colarbar with seperate axis
    cbax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel('Ratio Error Cp/u (-)', rotation=90)
    fig.suptitle('Ratio between errors in RWP phase speed and zonal wind for each lead day')
    #save figure for each hemisphere
    if lat_polar==90:
        fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_NH.png')
    else:
        fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_SH.png')
    plt.close(fig)

In [5]:
path ='/media/onno/Algemeen/Thesis/GFS_mean_season/'
seasonz = ['MAM','JJA','SON','DJF']
for season in seasonz:
    for i in range(1,11):
        
        file_error_u_wind = 'GFS_mean_error_u_wind_{}_day_{:02d}_globe.nc'.format(season,i)
        file_error_phasespeed = 'GFS_mean_error_phasespeed_{}_day_{:02d}_globe.nc'.format(season,i)
        error_u_wind = xr.open_dataset(path+file_error_u_wind,decode_times=False)
        error_phasespeed = xr.open_dataset(path+file_error_phasespeed,decode_times=False)
        error_u_wind = error_u_wind.where(np.abs(error_u_wind.u)>0.1)
        diff = (error_phasespeed.v /error_u_wind.u).to_dataset(name='ratio')
        diff.to_netcdf(path+'/ratio_cp_u/difference_error_phasespeed_u_wind_day_{:02d}_{}.nc'.format(i,season))


In [ ]:
path ='/media/onno/Algemeen/Thesis/GFS_mean_season/'
latz_polar=[-90,90]
seasonz = ['MAM','JJA','SON','DJF']
for season in seasonz:
    for lat_polar in latz_polar:
        fig,axz = plt.subplots(2,5,figsize=(20,10))
        for i in range(10):
            ax = axz.flat[i]
            ax.text(x=0.5,y=1.15,s='Day {}'.format(i),
               transform=ax.transAxes,horizontalalignment='center',fontsize=12)
            ds = xr.open_dataset(path+'/ratio_cp_u/difference_error_phasespeed_u_wind_day_{:02d}_{}.nc'.format(i+1,season),
                                decode_times=False)
            ds_cyclic, lon_cyclic = addcyclic(ds.ratio.values, ds.lon)
            lon,lat = np.meshgrid(lon_cyclic,ds.lat)

            m = Basemap(projection='ortho',lon_0=0,lat_0=lat_polar,resolution='c',ax=ax)
            im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
                       vmin=-2,vmax=2,levels=np.linspace(-2,2,81))
            m.drawcoastlines()
            m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
            m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1])
        fig.subplots_adjust(top=0.85,bottom=0.05,left=0.05,
                            right=0.9,hspace=0.25,wspace=0.25)
        cbax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
        fig.colorbar(im, cax=cbax)
        cbax.set_ylabel('Ratio Error Cp/u (-)', rotation=90)
        fig.suptitle('Ratio between errors in RWP phase speed and zonal wind for each lead day during {}'.format(season))
        if lat_polar==90:
            fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_{}_NH.png'.format(season))
        else:
            fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_{}_SH.png'.format(season))
        plt.close(fig)



In [23]:
latz_polar=[90]
path = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
variable='u_wind'
for lat_polar in latz_polar:
    fig,axz = plt.subplots(2,2,figsize=(20,10))
    j=0
    for i in range(10):
        if np.isin(i,[2,4,6,8])==False:
            continue
        ax = axz.flat[j]
        j+=1

        ax.text(x=0.5,y=1.15,s='Day {}'.format(i),
           transform=ax.transAxes,horizontalalignment='center',fontsize=12)
        ds = xr.open_dataset(path+'/ratio_cp_u/difference_error_phasespeed_u_wind_day_{:02d}.nc'.format(i+1),
                            decode_times=False)
#         ds = xr.open_dataset(path+'GFS_mean_error_{}_day_{:02d}_globe_mars.nc'.format(plot_dic[variable]['filename'],i+1),
#                             decode_times=False)
        ds_cyclic, lon_cyclic = addcyclic(ds.ratio.values, ds.lon)
        lon,lat = np.meshgrid(lon_cyclic,ds.lat)

        m = Basemap(lon_0=-95,lat_0=45,llcrnrlon=-160,llcrnrlat=25,urcrnrlon=-40,
                    urcrnrlat=70,resolution='c',ax=ax)
        im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
                   vmin=-2,vmax=2,levels=np.linspace(-2,2,81))
        m.drawcoastlines()
        m.drawcountries()
        m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
        m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1])
    fig.subplots_adjust(top=0.85,bottom=0.05,left=0.05,
                        right=0.9,hspace=0.25,wspace=0.25)
    cbax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
    fig.colorbar(im, cax=cbax)
    cbax.set_ylabel('Ratio Error Cp/u (-)', rotation=90)
    fig.suptitle('Ratio between errors in RWP phase speed and zonal wind for each lead day')
#     cbax.set_ylabel(plot_dic[variable]['label'], rotation=90)
#     fig.suptitle('{} Error in North America'.format(plot_dic[variable]['title']))
    sys.exit()
    if lat_polar==90:
        fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_NH.png')
    else:
        fig.savefig(path+'/ratio_cp_u/ratio_phasespeed_u_wind_SH.png')
    plt.close(fig)

SystemExit: 